In [1]:
import os
import os.path
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import  train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, roc_auc_score,
    classification_report, f1_score, roc_auc_score, recall_score, roc_curve
    )

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 500)
np.set_printoptions(
    edgeitems=5, linewidth=200, suppress=True, nanstr='NaN',
    infstr='Inf', precision=5
    )

RANDOM_STATE = 516
TRAIN_SIZE   = .70
TEST_SIZE    = 1 - TRAIN_SIZE


# fpath = "S:\\public\\Actuarial\\DSSG\\20170721_Materials\\mw.data"
# hdrs  = ["ID", "GENDER", "HEIGHT", "HAND_LENGTH", "FOREARM_LENGTH"]
# df    = pd.read_table(fpath, sep="\s+", names=hdrs)


#data_path = "https://gist.githubusercontent.com/jtrive84/c1e23acb2624733ada178260cc3c683c/raw/3cf7e81263de12727ac4d6b7391bf6766f942f2b/admissions.csv"
#data_path = "G:/Datasets/Binary_Response/Gold/gold.data"
# dfall = pd.read_csv(data_path)


# categorical_vars = ["rank"]
# continuous_vars  = ["gre", "gpa",]
# response         = "admit"


# Split data into train, validation and test cohorts. 
# dftrain, dftest0, ytrain, ytest0 = \
#     train_test_split(
#         dfall[categorical_vars + continuous_vars], 
#         dfall[response], test_size=TEST_SIZE, random_state=RANDOM_STATE
#         )

# # Split dftest0 into validation and test cohorts.
# dfvalidate, dftest, yvalidate, ytest = \
#     train_test_split(
#         dftest0, ytest0, test_size=.30, random_state=RANDOM_STATE
#         )


# # Recombine dftrain + ytrain, dfvalidate + yvalidate and dftest + ytest.
# dftrain = dftrain.join(ytrain).reset_index(drop=True)
# dfvalidate = dfvalidate.join(yvalidate).reset_index(drop=True)
# dftest = dftest.join(ytest).reset_index(drop=True)


In [1]:
"""
Using ColumnTransformer.

    - https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html
    - https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
    
Applies transformers to columns of an array or pandas DataFrame.


NOTES:
As implemented here, I used ColumnTransformer to pre-process the test and train data, while fitting
the model outside of the pipeline. This allows for calling `fit_transform` on the data,
as opposed to just `fit` as is typically used in examples that use Pipeline with an embedded 
estimator. This seems like a cleaner workflow to me, since typical workflows I've implemented
pre-process the data once, then fit a number of separate classifiers. 


Questions:

For pipelines with an embedded predictor, does `fit` do the same as `fit_transform`? Do we only
call `fit` on these pipelines (with embedded classifiers) since classifiers do not expose a 
`fit_transform` method?

Answer: Each object in the pipeline is assumed to have both fit and transform methods.
Therefore, they are both called on each transformer.

"""
import os
import os.path
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    confusion_matrix, accuracy_score, precision_score, roc_auc_score,
    classification_report, f1_score, roc_auc_score, recall_score
    )

pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 500)
np.set_printoptions(
    edgeitems=5, linewidth=200, suppress=True, nanstr='NaN',
    infstr='Inf', precision=5
    )

RANDOM_STATE = 516
TRAIN_SIZE   = .70
TEST_SIZE    = 1 - TRAIN_SIZE


# data_path = "G:\\Datasets\\Binary_Response\\Titanic\\Titanic.csv"
# dfinit = pd.read_csv(data_path)

data_path = "G:\\Datasets\\Books\\Wiley_Applied_Logistic_Regression\\NHANES.txt"
dfinit = pd.read_csv(data_path, sep="\t")

continuous_features  = [
    "AGE", 
    ]

categorical_features = [
    "GENDER", "CLASS",
    ]

response = "OBESE"

# Convert binary indicators to string fields so we can perform 
# our own onehot encoding and maintain the base levels of each.

# Redefine MARSTAT as currently married or not. 
dfinit["MARSTAT"] = dfinit["MARSTAT"].map(lambda v: 1 if v==1 else 0)



In [ ]:
for row in df.iterrows():
    
    if row["VIGRECEXR"] == row["OBESE"]:
        
        print("same")


In [40]:
dfinit["Label"] = dfinit["MARSTAT"]
df = dfinit.iloc[:5,-5:-1]
print(df)

dflist = [r for r in df.iterrows()]
[type(i) for i in dflist[0]]


   VIGRECEXR  MODRECEXR  SEDMIN  OBESE
0        1.0        1.0   480.0    0.0
1        1.0        0.0   240.0    0.0
2        1.0        1.0   240.0    1.0
3        0.0        1.0   720.0    0.0
4        1.0        1.0   240.0    0.0


[int, pandas.core.series.Series]

In [34]:
df


,VIGRECEXR,MODRECEXR,SEDMIN,OBESE,Label
0,1.0,1.0,480.0,0.0,1
1,1.0,0.0,240.0,0.0,0
2,1.0,1.0,240.0,1.0,0
3,0.0,1.0,720.0,0.0,1
4,1.0,1.0,240.0,0.0,0
5,1.0,0.0,60.0,0.0,1
6,1.0,1.0,540.0,0.0,0
7,0.0,0.0,480.0,0.0,0
8,1.0,1.0,30.0,1.0,1
9,1.0,1.0,NaN,0.0,1


In [29]:
l = [row[1] for row in df.iterrows()]


In [33]:
list(df.iterrows())[1]

(1, VIGRECEXR      1.0
 MODRECEXR      0.0
 SEDMIN       240.0
 OBESE          0.0
 Label          0.0
 Name: 1, dtype: float64)

In [26]:
for row in l:
    
    row = row[1]
    print(row)
    

VIGRECEXR      1.0
MODRECEXR      1.0
SEDMIN       480.0
OBESE          0.0
Label          1.0
Name: 0, dtype: float64
VIGRECEXR      1.0
MODRECEXR      0.0
SEDMIN       240.0
OBESE          0.0
Label          0.0
Name: 1, dtype: float64
VIGRECEXR      1.0
MODRECEXR      1.0
SEDMIN       240.0
OBESE          1.0
Label          0.0
Name: 2, dtype: float64
VIGRECEXR      0.0
MODRECEXR      1.0
SEDMIN       720.0
OBESE          0.0
Label          1.0
Name: 3, dtype: float64
VIGRECEXR      1.0
MODRECEXR      1.0
SEDMIN       240.0
OBESE          0.0
Label          0.0
Name: 4, dtype: float64
VIGRECEXR     1.0
MODRECEXR     0.0
SEDMIN       60.0
OBESE         0.0
Label         1.0
Name: 5, dtype: float64
VIGRECEXR      1.0
MODRECEXR      1.0
SEDMIN       540.0
OBESE          0.0
Label          0.0
Name: 6, dtype: float64
VIGRECEXR      0.0
MODRECEXR      0.0
SEDMIN       480.0
OBESE          0.0
Label          0.0
Name: 7, dtype: float64
VIGRECEXR     1.0
MODRECEXR     1.0
SEDMIN       30.0

In [28]:
row[1]

1.0

0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0


In [2]:
dfinit.columns = [i.upper() for i in dfinit.columns]




# If necessary, convert response to 0/1 with LabelBinarizer.
# lb = LabelBinarizer()
# df[response] = lb.fit_transform(df[response].values)


X = pd.get_dummies(
    dfinit[continuous_features + categorical_features],
    columns=categorical_features, drop_first=True
    )


y = dfinit[response].values


# Update categorical features to account for one hot encoding.
categorical_features = [i for i in X.columns if i not in continuous_features]

In [3]:
"""
Setup pre-processing pipeline. The implementation in this cell only handles
pre-processing and does not fit an estimator at the end. 
"""
# Split data into train-test cohorts.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)

continuous_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(missing_values=np.NaN, strategy="median", fill_value=0)),
    ("scaler" , StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(missing_values=np.NaN, strategy="constant", fill_value=0))
    ])

preprocessor = ColumnTransformer(transformers=[
    ("continuous" , continuous_transformer, continuous_features), 
    ("categorical", categorical_transformer, categorical_features)
    ])

pp = Pipeline(steps=[
    ("preprocessor", preprocessor), 
    ])

# Pass training data into preprocessing pipeline.
X_train = pp.fit(X_train)

# X_train can now be used to fit any number of estimators...

# lr = LogisticRegression()
# mdl1 = lr.fit(X_train)
# y_pred1 = mdl1.predict(X_test)

# clf = RandomForestClassifier()
# mdl2 = clf.fit(X_train)
# y_pred2 = mdl2.predict(X_test)


In [ ]:
"""
Performing grid search over specified parameter space, with estimator 
included as final step in pipeline.
"""
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30)

continuous_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(missing_values=np.NaN, strategy="median", fill_value=0)),
    ("scaler" , StandardScaler())
    ])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(missing_values=np.NaN, strategy="constant", fill_value=0))
    ])

preprocessor = ColumnTransformer(transformers=[
    ("continuous" , continuous_transformer, continuous_features), 
    ("categorical", categorical_transformer, categorical_features)
    ])

lr = Pipeline(steps=[
    ("preprocessor", preprocessor), 
    ("classifier"  , LogisticRegression(random_state=RANDOM_STATE))
    ])

param_grid = [{
    "classifier__fit_intercept":[True, False],
    "classifier__penalty"      :["elasticnet"],
    "classifier__solver"       :["saga"],
    "classifier__C"            :[0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "classifier__l1_ratio"     :np.arange(0, 1.10, .10),
    }]

gs = GridSearchCV(lr, param_grid, scoring="roc_auc", cv=5)

gs.fit(X_train, y_train)

In [11]:
lr_params = gs.best_params_
model     = gs.best_estimator_
y_pred    = model.predict(X_test)
y_prob    = model.predict_proba(X_test)[:,-1]


fpr, tpr, thresholds = roc_curve(y_test, y_prob,  pos_label=1)
# fpr2, tpr2, thresholds2 = roc_curve(y_test, lr_p_hat,  pos_label=1)
# fpr3, tpr3, thresholds3 = roc_curve(y_test, nb_p_hat,  pos_label=1)
# fpr4, tpr4, thresholds4 = roc_curve(y_test, knn_p_hat, pos_label=1)


# plt.plot(fpr1, tpr1, linewidth=2, label='Random Forest')
# plt.plot(fpr2, tpr2, linewidth=2, label='Logistic Regression')
# plt.plot(fpr3, tpr3, linewidth=2, label='Naive Bayes')
# plt.plot(fpr4, tpr4, linewidth=2, label='kNN')
# plt.plot([0,1], [0,1], '--')
# plt.axis([-.05, 1.05, -.05, 1.05])
# plt.xlabel('False Positive Rate (FPR)')
# plt.ylabel('True Positive Rate (TPR)')
# plt.grid(True)
# plt.legend(loc='lower right',prop={'size':20}, frameon=True)
# plt.show()


array([1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
     

In [ ]:
# Fit logistic regression model on training data.
lr =  LogisticRegression(C=1.0, solver="lbfgs")
lr.fit(X_train, y_train)

# Preprocess test data (only call transform on test data cohort).
X_test  = pp.transform(X_test)
y_pred = lr.predict(X_test)
y_prob = lr.predict_proba(X_test)

print("Accuracy : {}".format(accuracy_score(y_test, y_pred)))
print("Precision: {}".format(precision_score(y_test, y_pred, average="weighted")))
print("Recall   : {}".format(recall_score(y_test, y_pred, average="weighted")))
print("f1-score : {}".format(f1_score(y_test, y_pred, average="weighted")))
print("ROC-AUC  : {}".format(roc_auc_score(y_test, y_pred)))
print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
y_test


In [ ]:

X_train

In [ ]:
"""
Using BaseEstimator + TransformerMixin.

    - https://gist.github.com/amberjrivera/8c5c145516f5a2e894681e16a8095b5c
    
    - https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer.html#sphx-glr-auto-examples-compose-plot-column-transformer-py

"""
from sklearn.base import BaseEstimator, TransformerMixin

class AverageWordLengthExtractor(BaseEstimator, TransformerMixin):
    """Takes in dataframe, extracts road name column, outputs average word length"""

    def __init__(self):
        pass

    def average_word_length(self, name):
        """Helper code to compute average word length of a name"""
        return np.mean([len(word) for word in name.split()])

    def transform(self, df, y=None):
        """The workhorse of this feature extractor"""
        return df['road_name'].apply(self.average_word_length)

    def fit(self, df, y=None):
        """Returns `self` unless something different happens in train and test"""
        return self
    
    
    
# Unless you’re doing something more complicated where something different happens 
# in the training and testing phase (like when extracting n-grams), this is the 
# general pattern for a transformer:
class SampleExtractor(BaseEstimator, TransformerMixin):

    def __init__(self, vars):
        self.vars = vars  # e.g. pass in a column name to extract

    def transform(self, X, y=None):
        return do_something_to(X, self.vars)  # where the actual feature extraction happens

    def fit(self, X, y=None):
        return self  # generally does nothing
    
    


In [ ]:
# Using GridSearchCV within Pipeline

# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV

# https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html




In [ ]:
# Using FeatureUnion.

# https://scikit-learn.org/stable/modules/compose.html?highlight=featureunion#featureunion-composite-feature-spaces


# We have a slight problem: we only know how to add transformers in series, but what we need 
# to do is to add our average word length transformer in parallel with the n-gram extractor.
# For this, there is scikit-learn’s FeatureUnion class.
from sklearn.pipeline import Pipeline, FeatureUnion

pipeline = Pipeline([
    ('feats', FeatureUnion([
        ('ngram', ngram_count_pipeline), # can pass in either a pipeline
        ('ave', AverageWordLengthExtractor()) # or a transformer
    ])),
    ('clf', LinearSVC())  # classifier
])


# If you’re looking to do hyperparameter tuning (which I won’t explain here), pipelines 
# make that easy, as below:
from sklearn.grid_search import GridSearchCV

pg = {'clf__C': [0.1, 1, 10, 100]}

grid = GridSearchCV(pipeline, param_grid=pg, cv=5)
grid.fit(data_train, y_train)

grid.best_params_
# {'clf__C': 0.1}

grid.best_score_
# 0.702290076336




### Difference between Pipeline and make_pipeline

> https://stackoverflow.com/questions/40708077/what-is-the-difference-between-pipeline-and-make-pipeline-in-scikit


`Pipeline`:

- names are explicit, you don't have to figure them out if you need them;
- name doesn't change if you change estimator/transformer used in a step, e.g. if you replace `LogisticRegression()` with `LinearSVC()` you can still use `clf__C`.


`make_pipeline`:

- shorter and arguably more readable notation
- names are auto-generated using a straightforward rule (lowercase name of an estimator)\






## Machine Learning with sklearn JDTAQ


*Why do we call `fit_transform` on training data but only `transform` on test data?*

The reason you want to fit the scaler using only the training data is because you don't want to bias your 
model with information from the test data.



